In [25]:
import pandas as pd
import folium
from shapely.geometry import Point, Polygon
from sklearn.cluster import KMeans
from scipy.spatial import ConvexHull
import numpy as np


In [18]:
import pandas as pd
file_path = "data/0307.csv"
df = pd.read_csv(file_path)
df.head()

,元智大學 1132(CM254C),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,下載日期:2025/3/7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,點名日期,課號,課名,節次,學號,姓名,經度,緯度,簽到時間
3,20250307,1132-CM254C,機器學習應用,第 02 節,1101444,徐英豈,24.968098,121.26738,45723.37974
4,20250307,1132-CM254C,機器學習應用,第 02 節,1102455,江毓婷,24.970192,121.263435,45723.39526


In [19]:
# 重新設置標題行，並刪除不必要的行
df_cleaned = df.iloc[2:].reset_index(drop=True)
df_cleaned.columns = df_cleaned.iloc[0]  # 設置新標題
df_cleaned = df_cleaned[1:].reset_index(drop=True)  # 刪除舊標題行

# 重命名關鍵欄位
df_cleaned = df_cleaned.rename(columns={"姓名": "Name", "經度": "Latitude", "緯度": "Longitude", "簽到時間": "Check-in Time"})

# 轉換數據類型
df_cleaned["Latitude"] = pd.to_numeric(df_cleaned["Latitude"], errors="coerce")
df_cleaned["Longitude"] = pd.to_numeric(df_cleaned["Longitude"], errors="coerce")

# 轉換簽到時間（Excel 時間格式轉換）
df_cleaned["Check-in Time"] = pd.to_numeric(df_cleaned["Check-in Time"], errors="coerce")
df_cleaned["Check-in Time"] = pd.to_datetime(df_cleaned["Check-in Time"], unit="D", origin="1899-12-30")

df_cleaned

,點名日期,課號,課名,節次,學號,Name,Latitude,Longitude,Check-in Time
0,20250307,1132-CM254C,機器學習應用,第 02 節,1101444,徐英豈,24.968098,121.267380,2025-03-07 09:06:49.535999714
1,20250307,1132-CM254C,機器學習應用,第 02 節,1102455,江毓婷,24.970192,121.263435,2025-03-07 09:29:10.463999784
2,20250307,1132-CM254C,機器學習應用,第 02 節,1102514,王 杉,24.976917,121.255814,2025-03-07 09:18:27.648000311
3,20250307,1132-CM254C,機器學習應用,第 02 節,1102527,簡佳媛,25.021704,121.255318,2025-03-07 09:12:55.871999887
4,20250307,1132-CM254C,機器學習應用,第 02 節,1103002,林郁綺,24.970827,121.267715,2025-03-07 09:18:57.024000190
5,20250307,1132-CM254C,機器學習應用,第 02 節,1111733,徐欣儀,24.970751,121.267769,2025-03-07 09:10:22.080000000
6,20250307,1132-CM254C,機器學習應用,第 02 節,1111742,陳薇珊,24.970842,121.267746,2025-03-07 09:15:27.072000155
7,20250307,1132-CM254C,機器學習應用,第 02 節,1112214,張婷婷,24.970793,121.267754,2025-03-07 09:17:14.207999965
8,20250307,1132-CM254C,機器學習應用,第 02 節,1112513,羅亦婷,24.970839,121.267738,2025-03-07 09:10:45.407999723
9,20250307,1132-CM254C,機器學習應用,第 02 節,1112848,康妤暄,24.970444,121.267296,2025-03-07 09:25:32.736000259


In [20]:
import folium

# 設定地圖中心為所有學生簽到位置的平均值
map_center = [df_cleaned["Latitude"].mean(), df_cleaned["Longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=14)

locations = []
# 在地圖上添加標記
for _, row in df_cleaned.iterrows():
    popup_text = f"姓名: {row['Name']}<br>簽到時間: {row['Check-in Time']}"
    location = [row["Latitude"], row["Longitude"]]
    locations.append(location)
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# 自動調整地圖視窗範圍以包含所有標記
if locations:
    m.fit_bounds(locations)

# 保存地圖為 HTML 文件
html_file_path = "test.html"
m.save(html_file_path)

# 提供下載連結
html_file_path


'test.html'

In [21]:
from sklearn.cluster import KMeans
import folium
import matplotlib.pyplot as plt

# 擷取經緯度資料
coords = df_cleaned[["Latitude", "Longitude"]]

# 使用 KMeans 分群
kmeans = KMeans(n_clusters=3, random_state=0)
df_cleaned["Cluster"] = kmeans.fit_predict(coords)

# 建立 Folium 地圖
map_center = [coords["Latitude"].mean(), coords["Longitude"].mean()]
m_clustered = folium.Map(location=map_center, zoom_start=16)

# 設定每個群體的顏色
colors = ['red', 'green', 'blue']

# 加入標記到地圖
for _, row in df_cleaned.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        color=colors[row["Cluster"]],
        fill=True,
        fill_color=colors[row["Cluster"]],
        fill_opacity=0.7,
        popup=f"姓名: {row['Name']}<br>群體: {row['Cluster']}<br>簽到時間: {row['Check-in Time']}"
    ).add_to(m_clustered)

# 地圖調整至所有點的位置
m_clustered.fit_bounds(coords.values.tolist())

# 儲存為 HTML
html_clustered_path = "clustered_signins.html"
m_clustered.save(html_clustered_path)

html_clustered_path


'clustered_signins.html'

In [22]:
import folium
from shapely.geometry import Polygon

# 座標（已轉換為十進位格式）
coords = [
    (24.970306, 121.263250),
    (24.969944, 121.263333),
    (24.969972, 121.265611),
    (24.968972, 121.265944),
    (24.965083, 121.267139),
    (24.965083, 121.268278),
    (24.966583, 121.269583),
    (24.970972, 121.268778),
    (24.971194, 121.267999),
    (24.970999, 121.267111),
    (24.970583, 121.266000)
]

# 計算地圖中心
center_lat = sum(p[0] for p in coords) / len(coords)
center_lon = sum(p[1] for p in coords) / len(coords)

# 建立 Folium 地圖
m = folium.Map(location=[center_lat, center_lon], zoom_start=17)

# 繪製多邊形
folium.Polygon(
    locations=coords,
    color='blue',
    weight=2,
    fill=True,
    fill_color='blue',
    fill_opacity=0.4,
).add_to(m)

# 可選：標記每個點
for idx, (lat, lon) in enumerate(coords):
    folium.Marker(location=[lat, lon], popup=f'點 {idx+1}').add_to(m)

# 顯示地圖
m

In [29]:
from shapely.geometry import Point, Polygon 

# 地圖中心
center_lat = sum(p[0] for p in coords) / len(coords)
center_lon = sum(p[1] for p in coords) / len(coords)

# 校園範圍多邊形
campus_polygon = Polygon(coords)
df_cleaned["InCampus"] = df_cleaned.apply(
    lambda row: campus_polygon.contains(Point(row["Latitude"], row["Longitude"])), axis=1
)

# 對校內資料分群（假設第一週簽到大多在教室）
campus_points = df_cleaned[df_cleaned["InCampus"]].copy()
kmeans = KMeans(n_clusters=8, random_state=42)
campus_points["SubCluster"] = kmeans.fit_predict(campus_points[["Latitude", "Longitude"]])
classroom_cluster = campus_points["SubCluster"].value_counts().idxmax()
classroom_points = campus_points[campus_points["SubCluster"] == classroom_cluster][["Latitude", "Longitude"]].values

# 建立教室範圍多邊形（擴張凸包）
hull = ConvexHull(classroom_points)
hull_points = classroom_points[hull.vertices]

# 擴張教室範圍（每點向外平移一小段距離）
expansion_ratio = 0.0001  # 緯度/經度的微小擴張量
center = classroom_points.mean(axis=0)
expanded_hull_points = []
for point in hull_points:
    vector = point - center
    norm = np.linalg.norm(vector)
    unit_vector = vector / norm if norm != 0 else vector
    expanded_point = point + unit_vector * expansion_ratio
    expanded_hull_points.append(tuple(expanded_point))

classroom_polygon = Polygon(expanded_hull_points)

# 用擴張後教室範圍來重新判斷簽到位置
df_cleaned["LocationType"] = "Outside"
df_cleaned["InCampus"] = df_cleaned.apply(
    lambda row: classroom_polygon.contains(Point(row["Latitude"], row["Longitude"])), axis=1
)
df_cleaned.loc[df_cleaned["InCampus"], "LocationType"] = "InClassroom"
df_cleaned.loc[
    ~df_cleaned["InCampus"] & df_cleaned.apply(lambda row: campus_polygon.contains(Point(row["Latitude"], row["Longitude"])), axis=1),
    "LocationType"
] = "NearCampus"

# 建立地圖
campus_map_final = folium.Map(location=[center_lat, center_lon], zoom_start=17)

# 校園範圍多邊形
folium.Polygon(
    locations=coords,
    color='blue',
    weight=2,
    fill=True,
    fill_color='blue',
    fill_opacity=0.2,
).add_to(campus_map_final)

# 教室範圍多邊形
folium.Polygon(
    locations=expanded_hull_points,
    color='green',
    weight=2,
    fill=True,
    fill_color='green',
    fill_opacity=0.1,
    popup="教室推估範圍"
).add_to(campus_map_final)

# 顏色設定
location_colors = {
    "InClassroom": "green",
    "NearCampus": "yellow",
    "Outside": "red"
}

# 加入簽到點
for _, row in df_cleaned.iterrows():
    color = location_colors.get(row["LocationType"], "gray")
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"姓名: {row['Name']}<br>簽到時間: {row['Check-in Time']}"
    ).add_to(campus_map_final)

# 儲存 HTML
campus_map_final_path = "data/op.html"
campus_map_final.save(campus_map_final_path)

campus_map_final_path

'data/op.html'